In [ ]:
import cv2
import numpy as np
#얼국 인식용 xml 파일 
face_classifier = cv2.CascadeClassifier('haarcascade_frontalcatface.xml')

#전체 사진에서 얼굴 부위만 잘라 리턴
def face_extractor(img):
    #흑백처리 
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    #얼굴 찾기 
    faces = face_classifier.detectMultiScale(gray,1.3,5)
    #찾은 얼굴이 없으면 None으로 리턴 
    if faces is():
        return None
    #얼굴들이 있으면 
    for(x,y,w,h) in faces:
        #해당 얼굴 크기만큼 cropped_face에 잘라 넣기 
        #근데... 얼굴이 2개 이상 감지되면??
        #가장 마지막의 얼굴만 남을 듯
        cropped_face = img[y:y+h, x:x+w]
    #cropped_face 리턴 
    return cropped_face

#카메라 실행 
cap = cv2.VideoCapture(0)
#저장할 이미지 카운트 변수 
count = 0
while True:
    #카메라로 부터 사진 1장 얻기 
    ret, frame = cap.read()
    #얼굴 감지 하여 얼굴만 가져오기 
    if face_extractor(frame) is not None:
        count+=1
        #얼굴 이미지 크기를 200x200으로 조정 
        face = cv2.resize(face_extractor(frame),(200,200))
        #조정된 이미지를 흑백으로 변환 
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
        #faces폴더에 jpg파일로 저장 
        # ex > faces/user0.jpg   faces/user1.jpg ....
        file_name_path = 'faces/user'+str(count)+'.jpg'          
        cv2.imwrite(file_name_path,face)
        
        #화면에 얼굴과 현재 저장 개수 표시          
        cv2.putText(face,str(count),(50,50),cv2.FONT_HERSHEY_COMPLEX,1,(0,255,0),2)
        cv2.imshow('Face Cropper',face)
    else:
        print("Face not Found")
        pass

    if cv2.waitKey(1)==13 or count==100:
        break

cap.release()
cv2.destroyAllWindows()
print('Colleting Samples Complete!!!')

In [ ]:
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join
data_path = 'faces/'
#faces폴더에 있는 파일 리스트 얻기 
onlyfiles = [f for f in listdir(data_path) if isfile(join(data_path,f))]
#데이터와 매칭될 라벨 변수 
Training_Data, Labels = [], []
#파일 개수 만큼 루프 
for i, files in enumerate(onlyfiles):    
    image_path = data_path + onlyfiles[i]
    #이미지 불러오기 
    images = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    #이미지 파일이 아니거나 못 읽어 왔다면 무시
    if images is None:
        continue    
    #Training_Data 리스트에 이미지를 바이트 배열로 추가 
    Training_Data.append(np.asarray(images, dtype=np.uint8))
    #Labels 리스트엔 카운트 번호 추가 
    Labels.append(i)

#훈련할 데이터가 없다면 종료.
if len(Labels) == 0:
    print("There is no data to train.")
    exit()

#Labels를 32비트 정수로 변환
Labels = np.asarray(Labels, dtype=np.int32)
#모델 생성 
model = cv2.face.LBPHFaceRecognizer_create()
#학습 시작 
model.train(np.asarray(Training_Data), np.asarray(Labels))
print("Model Training Complete!!!!!")

In [ ]:
#########여기서부터 오류 ㅜㅜㅜ

cascade = 'haarcascade_frontalcatface.xml'

#고양이 얼굴 인식 검출기 생성
face_cascade = cv2.CascadeClassifier(cascade)

#얼굴 검출용 이미지
#실시간으로 받아오기

# 얼굴 검출 함수 
#사진 경로가 들어가면 -> 이미지가 녹색상자로 검출됨

# 얼굴 검출 함수
#이미지가 들어갓을때 어떻게 얼굴이 검출되는지
def detectCatFace(imgPath):
    # 이미지 불러오기 
    img = cv2.imread(imgPath, cv2.IMREAD_COLOR);
    # 회색으로 변경 
    grayImg = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # 얼굴 검출 
    faces = face_cascade.detectMultiScale(grayImg, scaleFactor=SF, minNeighbors=N, minSize=MS)
    # 검출된 얼굴 개수 출력 
    print("The number of images found is : " + str(len(faces)))    
    # 검출된 얼굴 위치에 녹색 상자그리기 
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
    # 검출된 얼굴(녹색 상자가 그려진) 이미지 데이터를 리턴
    return img   

In [13]:
import cv2
import os
'''
scale factor는 1에 가까울수록 인식율이 좋지만 
그만큼 느려짐(그만큼 많은  
'''
SF  = 1.02
'''
내부 알고리즘에서 최소한 검출된 횟수이상 되야 인식
0이면 무수한 오 검출이 되고
1이면 한번 이상 검출된 곳만 인식된다.
값이 높아질수록 오인식율은 줄지만 그만큼 
인식율이 떨어진다.
'''
N = 2
'''
검출하려는 이미지의 최소 사이즈
이 값보다 작은 이미지는 무시 
'''
MS=(100,100)

# 고양이 얼굴 인식용 haarcascade 파일 위치 
cascade = 'haarcascade_frontalcatface.xml'

# 고양이 얼굴 인식 cascade 생성 
face_cascade = cv2.CascadeClassifier(cascade)

# 얼굴 검출용 이미지 
catPic = 'cat3.png'

# 얼굴 검출 함수 
def detectCatFace(imgPath):
    # 이미지 불러오기 
    img = cv2.imread(imgPath, cv2.IMREAD_COLOR);
    # 회색으로 변경 
    
    grayImg = cv2.imread(imgPath,0)
    # 얼굴 검출 
    faces = face_cascade.detectMultiScale(grayImg, scaleFactor=SF, minNeighbors=N, minSize=MS)
    # 검출된 얼굴 개수 출력 
    print("The number of images found is : " + str(len(faces)))    
    # 검출된 얼굴 위치에 녹색 상자그리기 
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
    # 검출된 얼굴(녹색 상자가 그려진) 이미지 데이터를 리턴
    return img    

# 얼굴 검출 함수 호출 
img = detectCatFace(catPic)
# 검출된 이미지가 있다면 화면에 표시 
if len(img) != 0:
    cv2.imshow('Face',img)

    
# 아무키나 눌릴때까지 대기 
cv2.waitKey(0)

cv2.destroyWindow('Face')

The number of images found is : 4
